In [3]:
import os
import sys
import glob
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


%matplotlib inline
tf.__version__

'2.4.1'

In [4]:
DATASET_DIR = os.getcwd()
os.listdir(DATASET_DIR)

['Razpis.docx',
 'Pravilnik.docx',
 'cnn.ipynb',
 'Ct.ipynb',
 '.ipynb_checkpoints',
 'overview.txt']

## Plot to make sure

In [ ]:
def glob_list(path = 'normal'):
    imgs = []
    for img_path in glob.glob(DATASET_DIR + '/' + path + '/*'):
        imgs.append(mpimg.imread(img_path))
    return imgs

def plot(img):
    fig = plt.figure()
    fig.suptitle('normal')
    plt.imshow(normal_images[0], cmap='gray') 

def grid(x,y):
    f, axarr = plt.subplots(2,2)
    axarr[0,0].imshow(x[0])
    axarr[0,1].imshow(y[0])
    axarr[1,0].imshow(x[1])
    axarr[1,1].imshow(y[1])

normal_images = glob_list('normal')
covid_images = glob_list('covid')

plot(normal_images[0])
plot(covid_images[0])

In [ ]:
grid(normal_images,covid_images)

## Configs

In [91]:
IMG_W = 150
IMG_H = 150
CHANNELS = 3

INPUT_SHAPE = (IMG_W, IMG_H, CHANNELS)
N_CLASSES = 2
EPOCHS = 15
BATCH_SIZE = 12

log_dir = '/logs'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(validation_split = 0.3)


train_generator = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_H, IMG_W),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training')


validation_generator = datagen.flow_from_directory(
    DATASET_DIR, 
    target_size=(IMG_H, IMG_W),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle= False,
    subset='validation')

## NN

In [94]:
import tensorflow as tf
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Activation, Flatten, BatchNormalization,AvgPool2D,MaxPool2D, Dropout,Conv2DTranspose, Reshape, Lambda, LeakyReLU, UpSampling2D, concatenate

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint,TensorBoard

es = EarlyStopping(monitor = 'loss', mode = 'auto')
rl = ReduceLROnPlateau(monitor = 'val_accuracy', patience = 2)
check = ModelCheckpoint(filepath = '/checkpoints',monitor = 'accuracy', save_weights_only = False, save_freq='epoch')
tensorboard = TensorBoard(log_dir = log_dir, histogram_freq=10, write_graph=True, write_images=False)

cbs = [es, rl, check, tensorboard]

In [103]:
def compile_and_build(model, input_shape = INPUT_SHAPE):
    model.build(INPUT_SHAPE)
    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
    model.summary()

### Convolutional model

In [104]:
act = tf.nn.leaky_relu
model = Sequential([
    Conv2D(64, (3,3), input_shape = INPUT_SHAPE, activation=act),
    MaxPooling2D((2,2)),
    #Conv2D(32, (3,3), activation = act),
    #MaxPooling2D((2,2)),
    Conv2D(16,(3,3), activation = act),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(32, activation = act),
    Dense(1, activation = 'sigmoid')
])

compile_and_build(model)

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_251 (Conv2D)          (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_104 (MaxPoolin (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_252 (Conv2D)          (None, 72, 72, 16)        9232      
_________________________________________________________________
max_pooling2d_105 (MaxPoolin (None, 36, 36, 16)        0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 20736)             0         
_________________________________________________________________
dense_33 (Dense)             (None, 32)                663584    
_________________________________________________________________
dense_34 (Dense)             (None, 1)               

## AlexNet

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout, Activation
from keras.layers.normalization import BatchNormalization

def AlexNet(input_shape):
    
    X_input = Input(input_shape)
    
    X = Conv2D(96,(11,11),strides = 4,name="conv0")(X_input)
    X = BatchNormalization(axis = 3 , name = "bn0")(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max0')(X)
    
    X = Conv2D(256,(5,5),padding = 'same' , name = 'conv1')(X)
    X = BatchNormalization(axis = 3 ,name='bn1')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max1')(X)
    
    X = Conv2D(384, (3,3) , padding = 'same' , name='conv2')(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(384, (3,3) , padding = 'same' , name='conv3')(X)
    X = BatchNormalization(axis = 3, name = 'bn3')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(256, (3,3) , padding = 'same' , name='conv4')(X)
    X = BatchNormalization(axis = 3, name = 'bn4')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max2')(X)
    
    X = Flatten()(X)
    
    X = Dense(4096, activation = 'relu', name = "fc0")(X)
    
    X = Dense(4096, activation = 'relu', name = 'fc1')(X) 
    
    X = Dense(len(dummy_labels),activation='softmax',name = 'fc2')(X)
    
    model = Model(inputs = X_input, outputs = X, name='AlexNet')
    return model


### Autoencoder

In [106]:
H = 28
W = 28
C = 1

## Latent space
latent_dim = 128

## Building the autoencoder
inputs = Input(shape=(H, W, C), name="inputs")
x = inputs

x = Conv2D(32, (3, 3), padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x = MaxPool2D((2, 2))(x)

x = Conv2D(64, (3, 3), padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x = MaxPool2D((2, 2))(x)

x = Flatten()(x)
units = x.shape[1]
x = Dense(latent_dim, name="latent")(x)
x = Dense(units)(x)
x = LeakyReLU(alpha=0.2)(x)
x = Reshape((7, 7, 64))(x)

x = Conv2DTranspose(64, (3, 3), strides=2, padding="same")(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2DTranspose(1, (3, 3), strides=2, padding="same")(x)
x = BatchNormalization()(x)
x = Activation("sigmoid", name="outputs")(x)

outputs = x

autoencoder = Model(inputs, outputs)
autoencoder.compile(optimizer='rmsprop', loss='binary_crossentropy')
autoencoder.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_255 (Conv2D)          (None, 28, 28, 32)        320       
_________________________________________________________________
batch_normalization_109 (Bat (None, 28, 28, 32)        128       
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_108 (MaxPoolin (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_256 (Conv2D)          (None, 14, 14, 64)        18496     
_________________________________________________________________
batch_normalization_110 (Bat (None, 14, 14, 64)        256 

In [95]:
def autoenc_plot():
    n = 10
    plt.figure(figsize=(20, 4))
    for i in range(n):
      # display original
      ax = plt.subplot(2, n, i + 1)
      plt.imshow(x_test[i])
      plt.title("original")
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)

      # display reconstruction
      ax = plt.subplot(2, n, i + 1 + n)
      plt.imshow(decoded_imgs[i])
      plt.title("reconstructed")
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)
    plt.show()

### UNet

In [66]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x

In [83]:
def get_unet(input_img, n_filters = 16, dropout = 0.1, batchnorm = True):
    """Function to define the UNET Model"""
    # Contracting Path
    c1 = conv2d_block(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)
    
    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)
    
    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)
    
    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)
    
    # Expansive Path
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    
    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    
    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    
    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [90]:
input_img = Input((128, 128,1), name='img')
model = get_unet(input_img, n_filters=16, dropout=0.05, batchnorm=True)
model.compile(optimizer='rmsprop', loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d_212 (Conv2D)             (None, 128, 128, 16) 160         img[0][0]                        
__________________________________________________________________________________________________
batch_normalization_88 (BatchNo (None, 128, 128, 16) 64          conv2d_212[0][0]                 
__________________________________________________________________________________________________
activation_85 (Activation)      (None, 128, 128, 16) 0           batch_normalization_88[0][0]     
____________________________________________________________________________________________

## Training

In [ ]:
def train(model = model, restore=False, filename=model_dir, args=args):
    if restore: 
        model = load_model(filename)
    history = model.fit(train_generator, validation_data = val_generator,epochs=args.epochs,callbacks=callbacks,batch_size=args.batch_size, verbose=3)
    return history

In [ ]:
def plot(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # ACCURACY
    plt.plot(acc)
    plt.plot(val_acc)
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    # LOSS
    plt.plot(loss)
    plt.plot(val_loss)
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

## Hp tuning

In [ ]:
!ls logs/hparam_tuning
!rm -rf ./logs/
!rm -rf checkpoint
!ls

In [ ]:
%load_ext tensorboard

In [ ]:
from tensorboard.plugins.hparams import api as hp

HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([8, 64]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.3))
HP_LEARNING_RATE= hp.HParam('learning_rate', hp.Discrete([0.001, 0.0005, 0.0001]))
HP_OPTIMIZER=hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', 'rmsprop']))

METRIC_ACCURACY = 'accuracy'
log_dir ='\\logs\\fit\\' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
with tf.summary.create_file_writer(log_dir).as_default():
    hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT,  HP_OPTIMIZER, HP_LEARNING_RATE],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [ ]:
def create_model(hparams):
    model = tf.keras.Sequential([
        Dense(512,input_shape=[5]),
        Dense(128,activation='relu',kernel_regularizer=l1(1e-4)),
        BatchNormalization(),
        Dense(128,activation='relu'),
        Dropout(hparams[HP_DROPOUT]),
        Dense(64, activation='relu'),
        Dropout(hparams[HP_DROPOUT]),
        Dense(hparams[HP_NUM_UNITS], activation='relu'),
        Dense(16,activation='relu'),    
        Dense(1)
    ], name = 'rentals_model')
    optimizer = hparams[HP_OPTIMIZER]
    learning_rate = hparams[HP_LEARNING_RATE]
    
    if optimizer == "adam":
        optimizer = tf.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == "sgd":
        optimizer = tf.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer=='rmsprop':
        optimizer = tf.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        raise ValueError("Not an optimizer name: %r" % (optimizer_name,))
    
    model.compile(optimizer = optimizer, loss = Huber(), metrics=['RootMeanSquaredError'])
    return model
def train_and_evaluate(model, hparams):
    
    hp_tuning = hp.KerasCallback(log_dir, hparams),
    cbs = [tensorboard, hp_tuning]
    
    history = model.fit(x_train, y_train,validation_split=0.1,epochs=100, shuffle = True, callbacks = cbs,verbose=2)
    _, rmse = model.evaluate(x_test,y_test)
    return rmse, history

In [ ]:
model = create_model(hparams)
model.summary()

In [ ]:

def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        model = create_model(hparams)
        accuracy, history = train_and_evaluate(model, hparams)
        #accuracy= tf.reshape(tf.convert_to_tensor(accuracy), []).numpy()
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
    return history